# 导入模块

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time, datetime 

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean


In [10]:
pd.set_option('display.max_columns', None)
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [11]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 获取订单数据

In [12]:
dd_date = '2024-06'
sql5 = f'''
SELECT 
tprm.order_number '订单号', tprm.total_receivable '总应收', tprm.all_rental '总租金', tprm.all_deposit '总押金', tod.new_actual_money '买断价', (tod.new_actual_money-tprm.all_rental) '买断尾款', tprm.commodity_premium '溢价费', tprm.buyout_balance_payment '残值', om.user_mobile '手机号', tprm.search_time '下单月份', tprm.model_number '型号', tprm.specification '规格', tprm.order_create_time '下单时间', om.create_time '创建时间', tprm.rembursement_status '还款状态', 
tprm.paid_periods '已还期数', tprm.end_date '租期结束日期', tprm.order_finish_date '订单完成时间'
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.order_method, om.order_type, cc.channel_type_id
FROM db_digua_business.t_postlease_receivables_monitoring tprm
LEFT JOIN db_digua_business.t_order om ON tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
WHERE tprm.search_time='{dd_date}'
and tprm.platform_package != '租完即送'
and om.merchant_id not in (15,18,21,30,31,33,79,85,99,107,124, 171, 172)
'''

df5 = query(sql5)
# df5 = df_order5.copy()

# 渠道归属

In [13]:
df5.loc[:,"来源渠道"]=df5["channel_name"].fillna("未知渠道")

df5.loc[:,"归属渠道"]=df5.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"],x['channel_type_id'],x['order_type']),axis=1)

# 导入台账数据

In [14]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")
dfck = pd.merge(df_ck[['订单号', '采购金额']],df5,left_on="订单号", right_on="订单号", how='inner')
dfck = dfck[dfck.采购金额>0]
dfck.loc[:, '毛利'] = ((dfck.买断价+dfck.溢价费-dfck.采购金额)/dfck.采购金额).map(lambda x: format(x, '.2%'))

In [15]:
dfck_new = dfck.copy()
def chennel(dfck_new):
    dfck_new.loc[:, '内存'] = dfck_new.规格.str.split('：').str[1].str.split(' ').str[0]
    dfck_new.loc[:, '型号内存'] = dfck_new.型号+ '_' + dfck_new.内存
    dfck_new_group = dfck_new.groupby('型号内存').agg({'订单号': 'count', '采购金额': 'sum', '总应收': 'sum', '买断价': 'sum', '溢价费': 'sum', '总租金': 'sum', '总押金': 'sum', '买断尾款': 'sum', '残值': 'sum'}).rename(columns={'订单号': '数量'})
    dfck_new_group.loc[:, '毛利'] = ((dfck_new_group.买断价+dfck_new_group.溢价费-dfck_new_group.采购金额)/dfck_new_group.采购金额).map(lambda x: format(x, '.2%'))
    dfck_new_group.loc[:, '租售比'] = (dfck_new_group.总租金/dfck_new_group.总押金).map(lambda x: format(x, '.2%')) #(dfck_new_group.总租金/dfck_new_group.数量)/(dfck_new_group.总押金/dfck_new_group.数量)
    return dfck_new_group

In [16]:
dfck_new_zw = dfck_new[dfck_new.归属渠道=='芝麻租物']
dfck_new_group = chennel(dfck_new_zw)
dfck_new_fzw = dfck_new[dfck_new.归属渠道!='芝麻租物']
dfck_new_fgroup = chennel(dfck_new_fzw)

with pd.ExcelWriter(f'F:/需求/方全龙需求/订单_{dd_date}.xlsx', engine='xlsxwriter') as writer:
    # dfck.to_excel(writer, sheet_name=f'订单_{dd_date}', index=False)
    dfck_new_group.to_excel(writer, sheet_name=f'芝麻租物订单SKU_{dd_date}')
    dfck_new_fgroup.to_excel(writer, sheet_name=f'非芝麻租物订单SKU_{dd_date}')